In [ ]:
from absl import app
from collections.abc import Iterator
from typing import NamedTuple

import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
import optax
import tensorflow_datasets as tfds

from functools import partial

from typing import Dict
from typing_extensions import TypeAlias
from typing import Callable, List, Optional, Tuple



# Set default device to CPU for JAX
jax.config.update("jax_platform_name", "cpu")

In [ ]:
# Get devices if any
devices = jax.devices("cpu")
num_devices = len(devices)
print(f"Detected the following devices: {tuple(devices)}")

In [ ]:
class Batch(NamedTuple):
  image: np.ndarray  # [B, H, W, 1]
  label: np.ndarray  # [B]

class TrainingState(NamedTuple):
  params: hk.Params
  opt_state: optax.OptState

Metrics: TypeAlias = Dict[str, jnp.ndarray]

In [ ]:
metrics: Metrics = {
    "accuracy": jnp.array([0.9, 0.92, 0.85]),
    "loss": jnp.array([0.2, 0.15, 0.25])
}

In [ ]:
def load_dataset(
    split: str,
    *,
    shuffle: bool,
    batch_size: int,
) -> Iterator[Batch]:
  """Loads the MNIST dataset."""
  ds, ds_info = tfds.load("mnist:3.*.*", split=split, with_info=True)
  ds.cache()
  if shuffle:
    ds = ds.shuffle(ds_info.splits[split].num_examples, seed=0)
  ds = ds.repeat()
  ds = ds.batch(batch_size)
  ds = ds.map(lambda x: Batch(**x))
  return iter(tfds.as_numpy(ds))

In [ ]:
def main():
    def forward_fn(x: jax.Array) -> jax.Array:
        x = x.astype(jnp.float32) / 255.
        mlp = hk.Sequential([
            hk.Flatten(),
            hk.Linear(300), jax.nn.relu,
            hk.Linear(100), jax.nn.relu,
            hk.Linear(10),
        ])
        return mlp(x)
    
    def loss_fn(params: hk.Params, batch: NamedTuple):
        logits = network.apply(params, batch.image)
        labels = jax.nn.one_hot(batch.label, NUM_CLASSES)
        loss_value = -jnp.sum(labels * jax.nn.log_softmax(logits)) / batch.image.shape[0]

        preds = jax.nn.softmax(logits, axis=-1)
        metrics: Metrics = {
            "preds": preds,
        }

        return loss_value, metrics

    @partial(jax.pmap, devices=devices, axis_name='batch')
    def update(state: NamedTuple, batch: NamedTuple) -> Tuple[NamedTuple, Metrics]:
        (loss_value, metrics), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params, batch)
        grads = jax.lax.pmean(grads, axis_name='batch')
        updates, opt_state = optimizer.update(grads, state.opt_state)
        params = optax.apply_updates(state.params, updates)
        return TrainingState(params, opt_state), metrics
    
    @partial(jax.pmap, devices=devices, axis_name='batch')
    def evaluate(state: NamedTuple, batch: NamedTuple):
        logits = network.apply(state.params, batch.image)
        predictions = jnp.argmax(logits, axis=-1)
        accuracy = jnp.mean(predictions == batch.label)
        return jax.lax.pmean(accuracy, axis_name='batch')

    NUM_CLASSES = 10
    BATCH_SIZE_TRAIN = 64
    BATCH_SIZE_TEST = 10000

    print("Initializing network..")
    network = hk.without_apply_rng(hk.transform(forward_fn))

    print("Initializing data..")
    data_train = load_dataset("train", shuffle=True, batch_size=BATCH_SIZE_TRAIN)
    data_test = load_dataset("test", shuffle=False, batch_size=BATCH_SIZE_TEST)

    print("Initializing parameters..")
    params = network.init(jax.random.PRNGKey(seed=0), next(data_train).image)
    optimizer = optax.adam(learning_rate=1e-4)
    opt_state = optimizer.init(params)
    state = TrainingState(params, opt_state)

    # Replicate parameters and optimizer state across devices
    state = jax.device_put_replicated(state, devices)

    print("Training..")
    NUM_EPOCHS = 3
    for epoch in range(NUM_EPOCHS):
        print(f"--- Epoch {epoch+1}/{NUM_EPOCHS} ---")
        for i in range(100):
            batch = next(data_train)
            # Split batch for pmap
            # batch = jax.tree.map(lambda x: x.reshape((num_devices, -1) + x.shape[1:]), batch)
            batch = jax.device_put_replicated(batch, devices)
            state, metrics = update(state, batch)

        # Evaluate on test batch
        batch_test = next(data_test)
        batch_test = jax.device_put_replicated(batch_test, devices)
        # batch_test = jax.tree.map(lambda x: x.reshape((num_devices, -1) + x.shape[1:]), batch_test)
        accuracy = evaluate(state, batch_test)
        accuracy = jax.device_get(accuracy)
        print("Accuracy : ", accuracy.mean())

main()